In [4]:
metadata_directory = '../data/metadata'
import sys, os
sys.path.append(os.path.abspath('../'))

In [5]:
import pygad
import copy
import src

class FlowOptimizer:
    def __init__(self, packets, model_name, flow_id, max_fitness):
        self.packets = packets
        self.model_name = model_name
        self.flow_id = flow_id
        self.ga_instance = None
        self.max_fitness = max_fitness
        
    def apply_best_solution(self):
        best_solution, best_solution_fitness, _ = self.ga_instance.best_solution()
        best_operations = self.decode_operations(best_solution)
        # Stosujemy najlepsze operacje do pakietów
        self.best_modified_packets = self.apply_operations_to_packets(best_operations, self.packets, self.flow_id)
        # Możesz zwrócić więcej informacji o najlepszym rozwiązaniu, jeśli potrzebujesz
        return self.best_modified_packets, best_solution_fitness

    def on_generation(self, ga_instance):
        #current_population = ga_instance.population
        #print(f"Flow ID: {self.flow_id}, Pokolenie: {ga_instance.generations_completed}")
        #print("Aktualna populacja:")
        #for solution in current_population:
        #    print(solution)
        # Tworzymy wiadomość zawierającą interesujące nas informacje
        #message = f"Najlepszy wynik: {ga_instance.best_solution()[1]}\n"
        #print(message)  # Wyświetlamy informacje na konsoli
        
        # Zapisujemy te same informacje do pliku
        #with open(f"progress_for_flow_{self.flow_id}.txt", "a") as file:
        #    file.write(message)
        pass
            
    def on_fitness(self, ga_instance):
        ga_instance.plot_fitness()

    def fitness_function(self, ga_instance, solution, solution_idx):
        
        fitnesses = []
        target_fitnesses = []
        
        operations = self.decode_operations(solution)
        
        modified_packets = self.packets # always check nop
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        target_fitnesses.append(1.0 - predict_single_flow_target(self.model_name, sizing_stats, timing_stats))
        
        modified_packets = self.apply_operations_to_packets(operations[0], op_type = 'size_norm')
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        #print(sizing_stats)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        target_fitnesses.append(1.0 - predict_single_flow_target(self.model_name, sizing_stats, timing_stats))
        
        modified_packets = self.apply_operations_to_packets(operations[1], op_type = 'size_uni')
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        target_fitnesses.append(1.0 - predict_single_flow_target(self.model_name, sizing_stats, timing_stats))
        
        modified_packets = self.apply_operations_to_packets(operations[2], op_type = 'time_norm')
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        target_fitnesses.append(1.0 - predict_single_flow_target(self.model_name, sizing_stats, timing_stats))
        
        modified_packets = self.apply_operations_to_packets(operations[3], op_type = 'time_uni')
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        target_fitnesses.append(1.0 - predict_single_flow_target(self.model_name, sizing_stats, timing_stats))
        
        #print(fitnesses)
        max_fitness_value = max(fitnesses)
        max_fitness_index = fitnesses.index(max_fitness_value)
        #print(max_fitness_index)

        max_fitness_target = target_fitnesses[max_fitness_index]

        
        # Zapisz maksymalną wartość fitness i indeks, jeśli jest większa niż obecnie zapisana maksymalna wartość
        #print(self.max_fitness)
        if max_fitness_value > self.max_fitness[0][0]:
            #print(f'CONDITION max: {max_fitness_value}, prev: {self.max_fitness[0]}')
            self.max_fitness = [[max_fitness_value, max_fitness_target], self.decode_operations(solution), max_fitness_index]
            #print(f'self:{self.max_fitness[0]}')
        
        #print(max_fitness_value)
        #print(f'solution:{solution}')
        #print(f'finresses: {fitnesses}, solution:{self.decode_operations(solution)}')
        
        return max_fitness_value

    def optimize_for_flow(self):
        
        #with open(f"progress_for_flow_{self.flow_id}.txt", "w") as file:
        #    file.write("")  # Tylko czyszczenie zawartości pliku
            
        ga_instance = pygad.GA(num_generations=2,
                               num_parents_mating=1,
                               fitness_func=self.fitness_function,
                               sol_per_pop=3,                          
                               mutation_probability=0.7,
                               suppress_warnings= True,
     #                          initial_population=np.array([
    #        [4.14645349, 9.76164109, 1.67182642, 2.09448623, 1, 0],
   #         [2.80090477, 4.24612439, 1.41426283, 1.12520895, 2, 1],
  #          [1.31962389, 1.47082939, 2.15440462, 1.3226799, 0, 2]
 #       ]),
                               on_generation=self.on_generation,
                               num_genes=6,  # Przykład, dostosuj do ilości operacji
                               gene_space = [{'low': 1.1, 'high': 5.0},  # size normal
                                             {'low': 0.25, 'high': 5.0}, # packet size uniform
                                             {'low': 1.1, 'high': 3.0}, # time normal
                                             {'low': 0.4, 'high': 2.4}, # time uniform
                                            [0,1,2],  # direction
                                            [0,1,2]]) # focus point 

        
        ga_instance.run()
        return ga_instance, self.max_fitness

    def decode_operations(self, genotype):
        operations = []
        

        focus_point_map = {0: "start", 1: "middle", 2: "end"}
        focus_point = None
        focus_point_index = genotype[5] 
        focus_point = focus_point_map.get(focus_point_index, None)  

        size_norm = {
            'scaling_factor': genotype[0],
            'direction': genotype[4],
            'focus_point': focus_point
        }
        
        size_uni = {
            'scaling_factor': genotype[1],
            'direction': genotype[4],
            'focus_point': focus_point
        }
        
        time_norm = {
            'scaling_factor': genotype[2],
            'direction': genotype[4],
            'focus_point': focus_point
        }
        
        
        time_uni = {
            'scaling_factor': genotype[3],
            'direction': genotype[4],
            'focus_point': focus_point
        }  
            
            
        
        operations = [size_norm, size_uni, time_norm, time_uni]
        return operations 
        


    def apply_operations_to_packets(self, operation, op_type):

        #packets = copy.deepcopy(self.packets) # deep copy for testing
        #print(f'size: {prepare_size_stats(self.packets, self.flow_id)}')
        #print(f'time: {prepare_timing_stats(self.packets, self.flow_id)}')


        scaling_factor = operation['scaling_factor']
        direction = operation.get('direction')
        focus_point = operation.get('focus_point')
        

        if op_type == 'size_norm':
            packets = adjust_packet_size_deepcopy(self.packets, self.flow_id, direction, 'normal', scaling_factor, focus_point)
            
            
        elif op_type == 'size_uni':
            packets = adjust_packet_size_deepcopy(self.packets, self.flow_id, direction, 'uniform', scaling_factor, focus_point)
            
        elif op_type == 'time_norm':
            packets = apply_time_perturbation_with_focus_deepcopy(self.packets, self.flow_id, 'normal', focus_point, scaling_factor)
            
        elif op_type == 'time_uni':
            #print(f'przed: {prepare_timing_stats(self.packets, self.flow_id)}')
            packets = apply_time_perturbation_with_focus_deepcopy(self.packets, self.flow_id, 'uniform', focus_point, scaling_factor)
            #print(f'po: {prepare_timing_stats(packets, self.flow_id)}')
            
        #print(packets)
        return packets
    

            
        

    #print(optimizer.best_solution())
    


In [7]:
import os
import pickle

def load_object_pickle(filepath):
    try:
        with open(filepath, 'rb') as pickle_file:
            obj = pickle.load(pickle_file)
        return obj
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return None

def process_and_save_results(directory_path):
    files_processed = 0
    for filename in os.listdir(directory_path):
        if "20110817-bot" not in filename and "20110819-bot" not in filename and '20110810-neris' not in filename:
            if filename.endswith(".pkl"):
                filepath = os.path.join(directory_path, filename)
                print(f"Attempting to process: {filename}")
                obj = load_object_pickle(filepath)
                if obj is not None:
                    try:
                        for i in range(len(obj)):
                            list = obj[i][0][1][0]
                            file_key = filename.replace(".pkl", "").replace("_perturbed", "")
                            
                            # Nazwa pliku wyjściowego odpowiadająca file_key
                            output_filepath = f"{file_key}.txt"
                            
                            # Otwieranie pliku do zapisu wyników dla każdego pliku .pkl
                            with open(output_filepath, 'w') as output_file:
                                results_line = f"{file_key}: {list}\n"
                                output_file.write(results_line)
                                print(f"Successfully processed and saved: {file_key}")
                                files_processed += 1
                    except Exception as e:
                        print(f"Error processing data from {filename}: {e}")
                else:
                    print(f"Skipping file due to loading error: {filename}")
    print(f"Total files processed: {files_processed}")

# Przykład użycia
#directory_path = '../../../../../../Volumes/Extreme_SSD/data/metadata'  # Ścieżka do twojego katalogu
directory_path = '../data/metadata' 
process_and_save_results(directory_path)


Attempting to process: ga_rf_fitness_botnet-capture-20110815-rbot-dos_perturbed.pkl
Error loading ../data/metadata/ga_rf_fitness_botnet-capture-20110815-rbot-dos_perturbed.pkl: Can't get attribute 'FlowOptimizer' on <module '__main__'>
Skipping file due to loading error: ga_rf_fitness_botnet-capture-20110815-rbot-dos_perturbed.pkl
Attempting to process: ga_rf_fitness_botnet-capture-20110815-fast-flux-2_perturbed.pkl
Error loading ../data/metadata/ga_rf_fitness_botnet-capture-20110815-fast-flux-2_perturbed.pkl: Can't get attribute 'FlowOptimizer' on <module '__main__'>
Skipping file due to loading error: ga_rf_fitness_botnet-capture-20110815-fast-flux-2_perturbed.pkl
Attempting to process: ga_rf_fitness_botnet-capture-20110812-rbot_perturbed.pkl
Error loading ../data/metadata/ga_rf_fitness_botnet-capture-20110812-rbot_perturbed.pkl: Can't get attribute 'FlowOptimizer' on <module '__main__'>
Skipping file due to loading error: ga_rf_fitness_botnet-capture-20110812-rbot_perturbed.pkl
Atte

In [27]:
directory_path = '../data/metadata'  # Ścieżka do twojego katalogu
output_filepath = 'processed_results.txt'  # Plik wyjściowy z wynikami
process_and_save_results(directory_path, output_filepath)


Attempting to process: ga_rf_fitness_botnet-capture-20110815-rbot-dos_perturbed.pkl


Successfully processed and saved: ga_rf_fitness_botnet-capture-20110815-rbot-dos
Attempting to process: ga_rf_fitness_botnet-capture-20110815-fast-flux-2_perturbed.pkl
Successfully processed and saved: ga_rf_fitness_botnet-capture-20110815-fast-flux-2
Attempting to process: ga_rf_fitness_botnet-capture-20110812-rbot_perturbed.pkl
Successfully processed and saved: ga_rf_fitness_botnet-capture-20110812-rbot
Attempting to process: ga_rf_fitness_botnet-capture-20110816-sogou_perturbed.pkl
Successfully processed and saved: ga_rf_fitness_botnet-capture-20110816-sogou
Attempting to process: ga_rf_fitness_botnet-capture-20110816-donbot_perturbed.pkl
Successfully processed and saved: ga_rf_fitness_botnet-capture-20110816-donbot
Attempting to process: ga_rf_fitness_botnet-capture-20110816-qvod_perturbed.pkl
Successfully processed and saved: ga_rf_fitness_botnet-capture-20110816-qvod
Attempting to process: ga_rf_fitness_botnet-capture-20110815-fast-flux_perturbed.pkl
Successfully processed and sa

In [ ]:
import os
import pickle
import sys, os
sys.path.append(os.path.abspath('../'))

def load_object_pickle(filepath):
    """
    Loads an object from a specified filepath using dill.
    """
    with open(filepath, 'rb') as pickle_file:
        obj = pickle.load(pickle_file)
    return obj


# Funkcja do przetwarzania plików w katalogu
def process_pickle_files(directory_path):
    results = {}

    for filename in os.listdir(directory_path):
        print(filename)
        if filename.endswith(".pkl"):
            filepath = os.path.join(directory_path, filename)
            obj = load_object_pickle(filepath)
            # Zakładamy, że obj[1] to nasza tablica/tablice do ekstrakcji
            list1 = obj[1][0] if isinstance(obj[1][0], list) else [obj[1][0]]
            list2 = obj[1][1:] if len(obj[1]) > 1 else []

            file_key = filename.replace(".pki", "").replace("_perturbed", "")
            results[file_key] = [list1, list2]

    return results

NameError: name 'results' is not defined